In [ ]:
# grid search for coefficients in a weighted average ensemble
from sklearn.datasets.samples_generator import make_blobs
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from numpy import array
from numpy import argmax
from numpy import tensordot
from numpy.linalg import norm
from itertools import product

In [ ]:
# fit model on dataset
def fit_model(trainX, trainY):
    trainY_enc = to_categorical(trainY)
    # define model
    model = Sequential()
    model.add(Dense(25, input_dim=2, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',
                 metrics=['accuracy'])
    # fit model
    model.fit(trainX, trainY_enc, epochs=500, verbose=0)
    return model

In [ ]:
# make an ensemble prediction for multi-class classification
def ensemble_predictions(members, weights, testX):
    # make predictions
    yhats = [model.predict(testX) for model in members]
    yhats = array(yhats)
    # weighted sum across ensemble members
    summed = tensordot(yhats, weights, axes=((0), (0)))
    # argmax across classes
    result = argmax(summed, axis=1)
    return result

In [ ]:
# evaluate a specific number of members in an ensemble
def evaluate_ensemble(members, weights, testX, testY):
    # make prediction
    yhat = ensemble_predictions(members, weights, testX)
    # calculate accuracy
    return accuracy_score(testY, yhat)

In [ ]:
# normalize a vector to have unit norm
def normalize(weights):
    # calculate l1 vector norm
    result = norm(weights, 1)
    # check for a vector of all zeros
    if result == 0.0:
        return weights
    # return normalized vector (unit norm)
    return weights / result

In [ ]:
# grid search weights
def grid_search(members, testX, testY):
    # define weights to consider
    w = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    best_score, best_weights = 0.0, None
    # iterate all possible combinations (cartesian product)
    for weights in product(w, repeat=len(members)):
        # skip if all weights are equal
        if len(set(weights)) == 1:
            continue
        # hack, normalize weight vector
        weights = normalize(weights)
        # evaluate weights
        score = evaluate_ensemble(members, weights, testX, testY)
        if score > best_score:
            best_score, best_weights = score, weights
            print('> %s %.3f' % (best_weights, best_score))
    return list(best_weights)

In [ ]:
# generate 2d classification dataset
X, Y = make_blobs(n_samples=1100, centers=3, n_features=2,
                 cluster_std=2, random_state=2)
# split into train and test
n_train = 100
trainX, testX = X[:n_train, :], X[n_train:, :]
trainY, testY = Y[:n_train], Y[n_train:]

In [ ]:
# fit all models
n_members = 5
members = [fit_model(trainX, trainY) for _ in range(n_members)]
# evaluate each single model on the test set
testY_enc = to_categorical(testY)

In [ ]:
for i in range(n_members):
    _, test_acc = members[i].evaluate(testX, testY_enc, verbose=0)
    print('Model %d: %.3f' % (i+1, test_acc))

In [ ]:
# evaluate averaging ensemble (equal weights)
weights = [1.0 / n_members for _ in range(n_members)]
score = evaluate_ensemble(members, weights, testX, testY)

In [ ]:
score

In [ ]:
# grid search weights
weights = grid_search(members, testX, testY)
score = evaluate_ensemble(members, weights, testX, testY)